In [0]:
%sql
SELECT * FROM odag.bronze.CMS_AUDIT_INPUT_STG;

In [0]:
file_name = spark.sql("SELECT FILE_NAME FROM global_temp.status_tracker_gtv")
file_name_value = file_name.collect()[0]['FILE_NAME']

print(file_name_value)

In [0]:
trunc_report = spark.sql(f"""
SELECT
FIRST_NAME AS `Enrollee First Name -- Actual Length(50)`,
LAST_NAME AS `Enrollee Last Name -- Actual Length(50)`,
CONTRACT_ID AS `Contract Id -- Actual Length(5)`,
DATE_FORWARDED_TO_IRE AS `Date Forwarded To IRE -- Actual Length(10)`, 
SOURCE_INDICATOR AS `Source Indicator -- Actual Length(25)`,
DATE_FORMAT(current_timestamp(),'MM/dd/yyyy HH:mm:ss a') AS `Action_Timestamp`,
'{file_name_value}' AS `File Name`
FROM odag.bronze.CMS_AUDIT_INPUT_STG
WHERE
LENGTH(TRIM(FIRST_NAME)) > 50
OR LENGTH(TRIM(LAST_NAME)) > 50
OR LENGTH(TRIM(CONTRACT_ID)) > 5
OR LENGTH(TRIM(DATE_FORWARDED_TO_IRE)) > 10
OR LENGTH(TRIM(SOURCE_INDICATOR)) > 25;
""");
display(trunc_report)

In [0]:
trunc_report.write\
    .mode("overwrite")\
    .option("header","true")\
    .csv("abfs://aggregated-files@databricksdl7.dfs.core.windows.net/CMS_AUDIT_TRUNCATION_REPORT")

display(spark.read\
    .option("header","true")\
    .csv("abfs://aggregated-files@databricksdl7.dfs.core.windows.net/CMS_AUDIT_TRUNCATION_REPORT"))